# Example

In [1]:
import torch
from abc import ABC, abstractmethod


# абстрактный класс для сверточного слоя
class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        
    def set_kernel(self, kernel):
        self.kernel = kernel
        
    @abstractmethod
    def __call__(self, input_tensor):
        pass


# класс-обертка над torch.nn.Conv2d для унификации интерфейса
class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)
    
    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel
        
    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


# функция, создающая объект класса cls и возвращающая свертку от input_matrix
def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]
    
    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)
    
    return layer(input_matrix)


# Функция, тестирующая класс conv2d_cls.
# Возвращает True, если свертка совпадает со сверткой с помощью torch.nn.Conv2d.
def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])
    
    in_channels = kernel.shape[1]
    
    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)
    
    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)
    
    return torch.allclose(custom_conv2d_out, conv2d_out) \
        and (custom_conv2d_out.shape == conv2d_out.shape)

print(test_conv2d_layer(Conv2d))

True


# Exercise

In [29]:
import torch
from abc import ABC, abstractmethod


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)
    print ('conv2d_out.shape: ', conv2d_out.shape)
    print ('conv2d_out: \n', conv2d_out)
    return torch.allclose(custom_conv2d_out, conv2d_out) \
             and (custom_conv2d_out.shape == conv2d_out.shape)


# Сверточный слой через циклы.
class Conv2dLoop(ABCConv2d):
    def __call__(self, input_tensor):
        # Напишите в этом месте вычисление свертки с использованием циклов.
        # returns batch_size, out_channels, output_height, output_width
        output_shape = calc_out_shape(input_tensor.shape, self.out_channels, self.kernel_size, self.stride, 0)
        output_tensor = torch.zeros(output_shape)
        for batch in range(output_shape[0]): # number of batch
            print('batch = ', batch)
            for yo in range(output_shape[2]): # output height
                # print('yo = ', yo)
                for xo in range(output_shape[3]): # output weight
                    # print('xo = ', xo)
                    for filters in range(input_tensor.shape[1]): # number of input channels
                        # print('filter = ', filters)
                        for channel in range(output_shape[1]): # number of output channels
                            pre_tensor = input_tensor[batch, filters, yo*self.stride:yo*self.stride+self.kernel_size, 
                                                  xo*self.stride:xo*self.stride+self.kernel_size]
                            output_tensor[batch, channel, yo, xo] += (pre_tensor* self.kernel[channel][filters]).sum()
                            # print('channel = ', channel) 
            print('output_tensor calc : \n', output_tensor)       
        
        print('input_tensor.shape: ', input_tensor.shape) # batch_size, in_channels, input_height, input_width
        print('kernel.shape: ', self.kernel.shape)
        print('stride: ', self.stride)
        print('required output_tensor.shape: ', output_shape)
        print('output_tensor.shape: ', output_tensor.shape)
        # print('input_tensor: \n', input_tensor)
        # print('output_tensor: \n', output_tensor)
        
        return output_tensor

# Корректность реализации определится в сравнении со стандартным слоем из pytorch.
# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):

print(test_conv2d_layer(Conv2dLoop))

batch =  0
output_tensor calc : 
 tensor([[[[5252.]]],


        [[[   0.]]]])
batch =  1
output_tensor calc : 
 tensor([[[[ 5252.]]],


        [[[12596.]]]])
input_tensor.shape:  torch.Size([2, 3, 4, 4])
kernel.shape:  torch.Size([1, 3, 3, 3])
stride:  2
required output_tensor.shape:  (2, 1, 1, 1)
output_tensor.shape:  torch.Size([2, 1, 1, 1])
conv2d_out.shape:  torch.Size([2, 1, 1, 1])
conv2d_out: 
 tensor([[[[ 5252.]]],


        [[[12596.]]]], grad_fn=<ConvolutionBackward0>)
True
